# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Dec 22 2022 9:05AM,253663,10,0086263797,ISDIN Corporation,Released
1,Dec 22 2022 9:05AM,253663,10,0086263803,ISDIN Corporation,Released
2,Dec 22 2022 9:05AM,253663,10,0086263802,ISDIN Corporation,Released
3,Dec 22 2022 9:05AM,253663,10,0086263801,ISDIN Corporation,Released
4,Dec 22 2022 9:05AM,253663,10,0086263810,ISDIN Corporation,Released
5,Dec 22 2022 9:05AM,253663,10,0086263813,ISDIN Corporation,Released
6,Dec 22 2022 9:05AM,253663,10,0086263817,ISDIN Corporation,Released
7,Dec 22 2022 9:05AM,253663,10,0086263827,ISDIN Corporation,Released
8,Dec 22 2022 9:05AM,253663,10,0086263828,ISDIN Corporation,Released
9,Dec 22 2022 9:05AM,253663,10,0086264680,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
38,253659,Released,1
39,253660,Released,9
40,253661,Released,13
41,253662,Released,1
42,253663,Released,11


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
253659,NaN,NaN,1.0
253660,NaN,NaN,9.0
253661,NaN,NaN,13.0
253662,NaN,NaN,1.0
253663,NaN,NaN,11.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253570,0.0,0.0,23.0
253572,0.0,0.0,2.0
253574,0.0,1.0,1.0
253578,0.0,0.0,1.0
253584,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
253570,0,0,23
253572,0,0,2
253574,0,1,1
253578,0,0,1
253584,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253570,0,0,23
1,253572,0,0,2
2,253574,0,1,1
3,253578,0,0,1
4,253584,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,253570,,,23
1,253572,,,2
2,253574,,1,1
3,253578,,,1
4,253584,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation
11,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation
12,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation
25,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc."
34,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc"
35,Dec 22 2022 8:52AM,253658,12,Hush Hush
36,Dec 22 2022 8:51AM,253657,12,Hush Hush
37,Dec 22 2022 8:50AM,253654,12,Hush Hush
53,Dec 22 2022 8:48AM,253655,12,Hush Hush
54,Dec 22 2022 8:44AM,253653,10,Emerginnova


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation,,,11
1,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation,,,1
2,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation,,,13
3,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc.",,,9
4,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc",,,1
5,Dec 22 2022 8:52AM,253658,12,Hush Hush,,,1
6,Dec 22 2022 8:51AM,253657,12,Hush Hush,,,1
7,Dec 22 2022 8:50AM,253654,12,Hush Hush,,,16
8,Dec 22 2022 8:48AM,253655,12,Hush Hush,,,1
9,Dec 22 2022 8:44AM,253653,10,Emerginnova,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation,11,,
1,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation,1,,
2,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation,13,,
3,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc.",9,,
4,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc",1,,
5,Dec 22 2022 8:52AM,253658,12,Hush Hush,1,,
6,Dec 22 2022 8:51AM,253657,12,Hush Hush,1,,
7,Dec 22 2022 8:50AM,253654,12,Hush Hush,16,,
8,Dec 22 2022 8:48AM,253655,12,Hush Hush,1,,
9,Dec 22 2022 8:44AM,253653,10,Emerginnova,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation,11,,
1,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation,1,,
2,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation,13,,
3,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc.",9,,
4,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation,11.0,NaN,NaN
1,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation,1.0,NaN,NaN
2,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation,13.0,NaN,NaN
3,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc.",9.0,NaN,NaN
4,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Dec 22 2022 9:05AM,253663,10,ISDIN Corporation,11.0,0.0,0.0
1,Dec 22 2022 9:05AM,253662,10,ISDIN Corporation,1.0,0.0,0.0
2,Dec 22 2022 9:05AM,253661,10,ISDIN Corporation,13.0,0.0,0.0
3,Dec 22 2022 8:53AM,253660,10,"Methapharm, Inc.",9.0,0.0,0.0
4,Dec 22 2022 8:52AM,253659,12,"Uniderm USA, Inc",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,6087056,104.0,8.0,10.0
12,1268283,20.0,0.0,0.0
16,253578,1.0,0.0,0.0
17,760877,3.0,0.0,0.0
19,1014442,38.0,1.0,0.0
20,253590,1.0,0.0,0.0
21,253603,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,6087056,104.0,8.0,10.0
1,12,1268283,20.0,0.0,0.0
2,16,253578,1.0,0.0,0.0
3,17,760877,3.0,0.0,0.0
4,19,1014442,38.0,1.0,0.0
5,20,253590,1.0,0.0,0.0
6,21,253603,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,104.0,8.0,10.0
1,12,20.0,0.0,0.0
2,16,1.0,0.0,0.0
3,17,3.0,0.0,0.0
4,19,38.0,1.0,0.0
5,20,1.0,0.0,0.0
6,21,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,104.0
1,12,Released,20.0
2,16,Released,1.0
3,17,Released,3.0
4,19,Released,38.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,17,19,20,21
Status,,,,,,,
Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
Executing,8.0,0.0,0.0,0.0,1.0,0.0,0.0
Released,104.0,20.0,1.0,3.0,38.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,17,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Released,104.0,20.0,1.0,3.0,38.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,17,19,20,21
0,Completed,10.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Executing,8.0,0.0,0.0,0.0,1.0,0.0,0.0
2,Released,104.0,20.0,1.0,3.0,38.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()